# Package Download


In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

# LSTM
<img src="https://user-images.githubusercontent.com/1250095/40670602-b1d0211a-63a4-11e8-9abc-a5de1f90a547.png">
<img src="https://blog.kakaocdn.net/dn/NsPUk/btqSjABi258/JseUW3yFT1RIZI4C0LxONk/img.png">  

+ LSTM_Cell
    + input_size : input data의 size (embedding을 할 경우 embedding의 size이다)
    + hidden_size : hidden state의 차원 수
    + bias : 각 식의 존재하는 편향 값


+ if x 가 m*n의 차원이면 h차원가 맞게 weight를 곱하여 m*h의 차원으로 변경한다.
+ Weight가 x2x 4개 x2h 4개가 필요하므로 각각 한번에 4개를 만들어서 잘라서 사용
+ 데이터 example을 시계열 데이터로 사용 [batch, sequece, variable] 3차원 데이터


In [ ]:

from audioop import bias


class LSTM_Cell(nn.Module):
    def __init__(self,input_size,hidden_size, bias=True):
        super().__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size
        self.bias= bias
        # input gate, forget gate, out put, tanh 4개
        self.x2h=nn.Linear(input_size,4*hidden_size,bias=bias)
        self.h2h=nn.Linear(hidden_size,4*hidden_size,bias=bias)
        
    
    def forward(self,hidden_state,cell_state,x):
        
        # 다 더해서 나눠버림
        # x :[batch, variable]
        gates=self.x2h(x)+self.h2h(x)
        ingate, forgetgate, cellgate, outgate =gates.chunk(4,dim=1)
        
        ingate=F.sigmoid(ingate)
        outgate=F.sigmoid(outgate)
        forgetgate=F.sigmoid(forgetgate)
        cellgate=F.tanh(cellgate)
        
        cell_state= torch.mul(hidden_state,forgetgate) +torch.mul(ingate,cellgate)
        hidden_state= torch.mul(outgate,F.tanh(cell_state))
        
        return cell_state, hidden_state

# LSTM Layer
<img src=https://www.researchgate.net/publication/360097054/figure/fig5/AS:11431281080276414@1661220974865/The-structure-of-the-two-layer-LSTM-The-input-of-the-LSTM-is.png>

Layer가 두개 이상일 경우

hidden_state와 cell state가 2개가 존재하고 다음으로 넘어감  
그러므로 차원을 [batch, layer, hidden_dim]으로 다음 사이클에 넘길 필요가 있음 

In [ ]:
from re import L
from tkinter import Variable
from turtle import forward


class LSTM_Model(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim,layer_num=1, bias=True) :
        super().__init__()
        self.hidden_dim=hidden_dim
        self.lstm=LSTM_Cell(input_dim,hidden_dim)
        # Layer가 2개 이상 사용했을 때 사용하는 LSTM_Cell
        self.lstm_layer=LSTM_Cell(hidden_dim,hidden_dim)
        self.fc=nn.Linear(hidden_dim,output_dim)
        self.layer_num=layer_num
    def forward(self,x):
        # x :[batch,sequnce ,variable]
       
        #h0,c0 : hidden_state,cell_state 초기화 [batch,,layer,hidden_dim] 
        if torch.cuda.is_available:
            hidden_state=Variable(torch.zeros(x.size (0),self.layer_num,self.hidden_dim).cuda())
        else:
            hidden_state=Variable(torch.zeros(x.size (0),self.layer_num,self.hidden_dim))
        if torch.cuda.is_available:
            cell_state=Variable(torch.zeros(x.size (0),self.layer_num,self.hidden_dim).cuda())
        else:
            cell_state=Variable(torch.zeros(x.size (0),self.layer_num,self.hidden_dim))
        
        outs=[]
        
        if self.layer_num==1:
            hidden_state=hidden_state.squeeze()
            cell_state=cell_state.squeeze()
            for seq in range(x.size(1)):
                cell_state,hidden_state = self.lstm(cell_state,hidden_state,x[:,seq,:])
                outs.append(hidden_state)
        else:
            for seq in range(x.size(1)):
                cell_state[:,0,:],hidden_state[:,0,:]=LSTM_Cell(cell_state,hidden_state,x[:,seq,:])
                for layer in range(self.layer_num-1):
                    cell_state[:,layer+1,:],hidden_state[:,layer+1,:]=self.lstm_layer(cell_state[:,layer+1,:],hidden_state[:,layer+1,:],hidden_state[:,layer,:])
                outs.append(hidden_state[:,-1,:])
                
        
        out =outs[-1]
        out= self.fc(out)
        return out
        